In [1]:
from torchmetrics import MeanSquaredError, SSIM
import os

files_to_remove = os.listdir("./output/")
for file_to_remove in files_to_remove:
    os.remove(f"./output/{file_to_remove}")

import matplotlib.pyplot as plt
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
import torch, torchvision
import detectron2
import json
import ast
import tensorboard, os
from detectron2.utils.logger import setup_logger

setup_logger()
import numpy as np
import pandas as pd
import wandb
import os, json, cv2, random
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from tqdm import tqdm
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

np.random.seed(42)
random.seed(42)
torch.manual_seed(42)

In [2]:
# Version
data = pd.read_csv("./download/Data.csv").sample(frac=1)
idx = 0
# Loading Data
def load_data(data=data, test=False):
    if test is True:
        if "data.npy" in os.listdir("./"):
            data = np.load("./data.npy", allow_pickle=True)
            data = data[:125]
            print(len(data))
            return data
    if "data.npy" in os.listdir("./"):
        data = np.load("./data.npy", allow_pickle=True)
        print(len(data))
        return data
    new_data = []
    for idx in tqdm(range(len(data))):
        record = {}
        info = data.iloc[idx]
        height, width = cv2.imread("./download/Img/" + info["Path"]).shape[:2]
        xmin, ymin, xmax, ymax = info["XMin"], info["YMin"], info["XMax"], info["YMax"]
        xmin = round(xmin * width)
        xmax = round(xmax * width)
        ymin = round(ymin * height)
        ymax = round(ymax * height)
        record["file_name"] = "./download/Img/" + info["Path"]
        record["height"] = height
        record["width"] = width
        record["cateogry_id"] = 0
        objs = [
            {
                "bbox": [xmin, ymin, xmax, ymax],
                "bbox_mode": BoxMode.XYXY_ABS,
                "category_id": 0,
            }
        ]
        record["image_id"] = idx
        record["annotations"] = objs
        new_data.append(record)
    np.random.shuffle(new_data)
    np.save("data.npy", new_data)
    return new_data

In [3]:
# Config
labels = ["Card"]
# Adding the data
DatasetCatalog.register("data", lambda: load_data())
MetadataCatalog.get("data").set(thing_classes=labels)
metadata = MetadataCatalog.get("data")
DatasetCatalog.register("test", lambda: load_data(test=True))
MetadataCatalog.get("test").set(thing_classes=labels)
metadata_test = MetadataCatalog.get("test")

In [4]:
from detectron2.utils.logger import setup_logger

setup_logger()
model = f"COCO-Detection/faster_rcnn_R_101_DC5_3x.yaml"
torch.cuda.empty_cache()
wandb.init(project="Find-Card", name="Final")
torch.cuda.empty_cache()
cfg = get_cfg()
torch.cuda.empty_cache()
cfg.merge_from_file(model_zoo.get_config_file(model))
torch.cuda.empty_cache()
cfg.DATASETS.TRAIN = ("data",)
torch.cuda.empty_cache()
cfg.DATASETS.TEST = ()
torch.cuda.empty_cache()
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model)
torch.cuda.empty_cache()
cfg.SOLVER.MAX_ITER = 500
torch.cuda.empty_cache()
cfg.TEST.EVAL_PERIOD = 500
cfg.SOLVER.BASE_LR = 0.00025
torch.cuda.empty_cache()
cfg.SOLVER.STEPS = []
torch.cuda.empty_cache()
cfg.SOLVER.IMS_PER_BATCH = 2
torch.cuda.empty_cache()
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(labels)
torch.cuda.empty_cache()
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
torch.cuda.empty_cache()
trainer = DefaultTrainer(cfg)
torch.cuda.empty_cache()
trainer.resume_or_load(resume=False)
torch.cuda.empty_cache()

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2021-10-27 22:50:59.255048: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


[10/27 22:51:08 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): ResNet(
    (stem): BasicStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv1): Conv2d(
          64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv2): Conv2d(
          64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv3): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
 

2021-10-27 22:51:09.128307: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Some model p

In [ ]:
trainer.train()

[10/27 22:51:12 d2.engine.train_loop]: Starting training from iteration 0
[10/27 22:51:23 d2.utils.events]:  eta: 0:04:11  iter: 19  total_loss: 1.464  loss_cls: 0.8388  loss_box_reg: 0.4272  loss_rpn_cls: 0.1113  loss_rpn_loc: 0.02705  time: 0.5558  data_time: 0.0160  lr: 9.7405e-06  max_mem: 4786M
[10/27 22:51:36 d2.utils.events]:  eta: 0:04:28  iter: 39  total_loss: 1.123  loss_cls: 0.6418  loss_box_reg: 0.3108  loss_rpn_cls: 0.1876  loss_rpn_loc: 0.01122  time: 0.5986  data_time: 0.0026  lr: 1.9731e-05  max_mem: 4786M
[10/27 22:51:48 d2.utils.events]:  eta: 0:04:16  iter: 59  total_loss: 1.172  loss_cls: 0.4395  loss_box_reg: 0.5184  loss_rpn_cls: 0.1096  loss_rpn_loc: 0.02679  time: 0.6030  data_time: 0.0026  lr: 2.972e-05  max_mem: 4786M
[10/27 22:52:01 d2.utils.events]:  eta: 0:04:10  iter: 79  total_loss: 0.9415  loss_cls: 0.3384  loss_box_reg: 0.437  loss_rpn_cls: 0.06635  loss_rpn_loc: 0.03756  time: 0.6070  data_time: 0.0026  lr: 3.9711e-05  max_mem: 4786M
[10/27 22:52:13 d2

In [ ]:
torch.cuda.empty_cache()
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.75
torch.cuda.empty_cache()
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
torch.cuda.empty_cache()
predictor = DefaultPredictor(cfg)
torch.cuda.empty_cache()
cfg.MODEL.WEIGHTS = "./output/model_final.pth"
cfg.SOLVER.SCORE_THRESH_TEST = 0.75
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("test", output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "test")
metrics = inference_on_dataset(predictor.model, val_loader, evaluator)
wandb.log(metrics)

In [ ]:
torch.cuda.empty_cache()
logs = open("./output/metrics.json", "r").read().split("\n")
for log in tqdm(range(len(logs))):
    try:
        res = ast.literal_eval(logs[log])
        wandb.log(res)
    except:
        pass

In [ ]:
for img in os.listdir("./test_imgs/"):
    torch.cuda.empty_cache()
    v = Visualizer(cv2.imread(f"./test_imgs/{img}")[:, :, ::-1], metadata=metadata)
    torch.cuda.empty_cache()
    v = v.draw_instance_predictions(
        predictor(cv2.imread(f"./test_imgs/{img}"))["instances"].to("cpu")
    )
    torch.cuda.empty_cache()
    v = v.get_image()[:, :, ::-1]
    torch.cuda.empty_cache()
    plt.figure(figsize=(24, 12))
    torch.cuda.empty_cache()
    plt.imshow(v)
    torch.cuda.empty_cache()
    plt.savefig(f"./preds/{img}")
    torch.cuda.empty_cache()
    plt.close()
    torch.cuda.empty_cache()
    wandb.log({f"Img/{img}": wandb.Image(cv2.imread(f"./preds/{img}"))})

In [ ]:
info = data.iloc[589]
print(info)
img = cv2.imread("./download/Img/" + info["Path"])
height, width = cv2.imread("./download/Img/" + info["Path"]).shape[:2]
xmin, ymin, xmax, ymax = info["XMin"], info["YMin"], info["XMax"], info["YMax"]
xmin = round(xmin * width)
xmax = round(xmax * width)
ymin = round(ymin * height)
ymax = round(ymax * height)
x = xmin
y = ymin
w = xmax - xmin
h = ymax - ymin
preds = predictor(img)
target = torch.tensor([xmin, ymin, xmax, ymax])

lowest_rmse = 0
r_mean_squared_error = MeanSquaredError(squared=False)
preds_new = preds['instances'].__dict__['_fields']['pred_boxes'].__dict__['tensor']
for pred_i in range(len(preds)):
    pred = preds_new[pred_i]
    if r_mean_squared_error(pred.to('cpu'), target) > lowest_rmse:
        lowest_rmse = r_mean_squared_error(pred.to('cpu'), target)

lowest_mse = 0
mean_squared_error = MeanSquaredError(squared=True)
preds_new = preds['instances'].__dict__['_fields']['pred_boxes'].__dict__['tensor']
for pred_i in range(len(preds)):
    pred = preds_new[pred_i]
    if mean_squared_error(pred.to('cpu'), target) > lowest_mse:
        lowest_mse = mean_squared_error(pred.to('cpu'), target)

wandb.finish()